In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rc('axes', unicode_minus=False)
%matplotlib inline

np.random.seed(123)

In [ ]:
train = pd.read_csv('data/job_train.csv')
test = pd.read_csv('data/job_test.csv')
df = pd.concat([train,test])
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14668 entries, 0 to 14667
Columns: 109 entries, STU_ID to X1STU30OCC2
dtypes: float64(36), int64(73)
memory usage: 12.2 MB


In [ ]:
df.X1STU30OCC2.nunique()

21

In [ ]:
df.X1STU30OCC2.value_counts()

29.0    3479
27.0    2088
19.0     844
17.0     828
23.0     657
25.0     616
33.0     478
55.0     461
11.0     369
39.0     286
15.0     280
49.0     254
35.0     232
13.0     217
21.0     165
47.0     134
53.0     114
41.0      88
51.0      69
31.0      43
45.0      38
Name: X1STU30OCC2, dtype: int64

In [ ]:
df = df.where(~df.isin([-9,-8,-7]), np.nan)


In [ ]:
null_check = df.isnull().sum()

In [ ]:
sum_df = df.describe().T.iloc[:,3:]
sum_df['null_check'] = null_check
sum_df.to_csv('data/sum_df.csv')

In [ ]:
sum_df 

,min,25%,50%,75%,max,null_check
STU_ID,10001.00,16317.75,22661.50,28909.75,35205.00,0
X1SEX,1.00,1.00,2.00,2.00,2.00,0
X1RACE,1.00,5.00,8.00,8.00,8.00,0
X1HISPANIC,0.00,0.00,0.00,0.00,1.00,0
X1WHITE,0.00,0.00,1.00,1.00,1.00,118
...,...,...,...,...,...,...
X1COUPERCOU,-6.43,-0.69,0.26,1.02,1.15,1574
X1COUPERPRI,-5.51,-0.68,0.20,1.10,1.10,1912
X1AQSTAT,1.00,1.00,1.00,1.00,8.00,0
X1AQDATE,200910.00,200911.00,201001.00,201001.00,201001.00,746


In [ ]:
##nullcheck 어떻게 처리할지 정해야함

nullp = pd.read_csv('data/null_pre.csv')
nullp = nullp.dropna().reset_index(drop=True)
nullp = nullp.set_index('variables')['null_pre']
nullp

variables
STU_ID         0.00
X1SEX          0.00
X1RACE         0.00
X1HISPANIC     0.00
X1WHITE        0.00
               ... 
X1COUPERTEA    0.03
X1COUPERCOU    0.26
X1COUPERPRI    0.20
X1AQSTAT       0.00
X1STU30OCC2    0.00
Name: null_pre, Length: 103, dtype: float64

In [ ]:
#csv에 작성한 null값을 읽어서 채움
df = df[nullp.index]
df = df.fillna(nullp)

In [ ]:
df.isnull().sum().sum()

0

In [ ]:
df['birthyear'] = (df['X1STDOB']/100).astype('int')
df['birthyear'] = (df.birthyear % 1990).astype('int')
df['birthmonth'] = (df['X1STDOB']%100).astype('int')

In [ ]:
test = df[df.X1STU30OCC2==0].reset_index(drop=True)
test = test.drop(columns = ['X1STU30OCC2'])
train = df[df.X1STU30OCC2!=0].reset_index(drop=True)

In [ ]:
idx2job = train['X1STU30OCC2'].value_counts().reset_index().astype('int')
idx2job = idx2job[['index']]
idx2job.columns = ['job']
idx2job.to_csv('data/idx2job.csv')
idx2job = idx2job['job'].to_dict()
job2idx = {v:i for i,v in idx2job.items()}

In [ ]:
train['X1STU30OCC2'] = train['X1STU30OCC2'].replace(job2idx).astype('int')

In [ ]:
train.to_csv('data/train_ver02.csv', index=False)
test.to_csv('data/test_ver02.csv', index=False)

In [ ]:
train.shape

(11740, 105)

In [ ]:
test.shape

(2928, 104)